## Création d'un utilisateur

Créer un utilisateur et les clés nécessaire au bon fonctionnement du chiffrement

In [ ]:
from key_management.user_enrollment import enroll_user

if __name__ == "__main__":
    username = input("Entrez le nom d'utilisateur pour l'enrôlement : ")
    enroll_user(username)

## Création des S-boxes

Afin de faire les permutations nécessaires dans COBRA, nous devons créer des S-boxes afin de permettre de mélanger les caractères de notre par un autre

In [3]:
from pathlib import Path
from utils.sboxe import *
from storage.sboxes_storage import save_s_boxes_to_file

PATH = Path("encryption") / "s_boxes.json"

if __name__ == "__main__":
    # Génération de 4 S-Boxes de taille 16 (pour blocs de 4 bits)
    s_boxes = generate_multiple_s_boxes(num_s_boxes=4, size=16)

    # Affichage des S-Boxes
    for i, s_box in enumerate(s_boxes):
        print(f"S-Box {i+1} : {s_box}")

    save_s_boxes_to_file(s_boxes, filename=PATH)

S-Box 1 : [6, 3, 11, 10, 0, 4, 1, 8, 14, 15, 9, 5, 2, 13, 12, 7]
S-Box 2 : [11, 10, 7, 13, 9, 2, 8, 5, 15, 4, 12, 0, 3, 6, 14, 1]
S-Box 3 : [5, 11, 3, 10, 7, 4, 0, 13, 15, 9, 6, 2, 12, 8, 14, 1]
S-Box 4 : [8, 11, 1, 6, 12, 10, 4, 15, 0, 7, 3, 13, 9, 14, 5, 2]
S-Boxes sauvegardées dans le fichier : encryption\s_boxes.json


## Fonctions Présentent dans encryption/cobra_cipher.py

Test des fonctions dans cobra cipher

In [12]:
from encryption.cobra_cipher import *

# Fonction principale pour le chiffrement
def encrypt_message(message, key, block_size=16):
    blocks = text_to_blocks(message, block_size)
    key_block = key_to_block(key, block_size)
    encrypted_blocks = [add_round_key(block, key_block) for block in blocks]
    return encrypted_blocks

# Déchiffrement (inverse de add_round_key est le XOR avec la même clé)
def decrypt_blocks(encrypted_blocks, key, block_size=16):
    key_block = key_to_block(key, block_size)
    decrypted_blocks = [add_round_key(block, key_block) for block in encrypted_blocks]
    decrypted_message = "".join("".join(chr(x) for x in block) for block in decrypted_blocks)
    # Retirer le padding
    padding_length = ord(decrypted_message[-1])
    return decrypted_message[:-padding_length]

# Afficher les blocs chiffrés en texte clair (illisible humainement mais converti en str)
def encrypted_blocks_to_text(encrypted_blocks):
    return "".join("".join(chr(x) for x in block) for block in encrypted_blocks)

# Tester la fonction
message = "Hello, World! This is a test message."
key = "Monmdp1234"

# Chiffrement
encrypted_blocks = encrypt_message(message, key)
encrypted_text = encrypted_blocks_to_text(encrypted_blocks)

print("Message original :", message)
print("Blocs chiffrés (en octets) :", encrypted_blocks)
print("Message chiffré en texte brut :", encrypted_text)

# Déchiffrement
decrypted_message = decrypt_blocks(encrypted_blocks, key)
print("Message déchiffré :", decrypted_message)

Message original : Hello, World! This is a test message.
Blocs chiffrés (en octets) : [array([  5,  10,   2,   1,  11,  92,  17, 101,  92,  70, 108, 100,  33,
        32,  84, 104], dtype=uint8), array([ 36,  28,  78,   4,  23,  80,  80,  18,  71,  81, 115, 116,  32,
       109, 101, 115], dtype=uint8), array([ 62,  14,   9,   8,  74, 123,  58,  57,  56,  63,  11,  11,  11,
        11,  11,  11], dtype=uint8)]
Message chiffré en texte brut : 
\e\Fld! Th$NPPGQst mes>J{:98?
Message déchiffré : Hello, World! This is a test message.
